In [ ]:
import pandas as pd
import rdflib
from rdflib import URIRef, OWL, Literal
from owlready2 import get_ontology
from tqdm import tqdm
from glob import glob
import logging
import os
from sklearn.model_selection import train_test_split
from src.utils import *


In [ ]:
dataset_name = 'OWL2DL-1'

**Filter unnecessary triples from g and i**

In [ ]:
ontology = get_ontology(f'datasets/{dataset_name}.owl').load()
subject_resources = list(ontology.individuals())
named_individuals = [URIRef(x.iri) for x in subject_resources]
print(f'# Subject-Resources: {len(subject_resources)}')

In [ ]:
def filter_inferred_triples(input_path, inferred_path, filtered_inferred_path):  
    g = rdflib.Graph()
    g.parse(input_path)  
    g_inferred = rdflib.Graph()
    g_inferred.parse(inferred_path)

    g_inferred_filtered = rdflib.Graph()
    for triple in g_inferred: 
        if (#(triple[0] in named_individuals or triple[2] in named_individuals) and 
            not isinstance(triple[2], Literal) and                           
            triple[2] != OWL.Thing and                                 
            triple not in g):
            g_inferred_filtered.add(triple)
    g_inferred_filtered.serialize(filtered_inferred_path, format="turtle")

In [ ]:
for input_graph_path in tqdm(sorted(glob('datasets/inferred_graphs/' + "*"))):
    file = input_graph_path.replace('datasets/inferred_graphs\\', '').replace('.xml', '')
    filter_inferred_triples('datasets/input_graphs/' + file + '.ttl', 
                            input_graph_path, 
                            'datasets/inferred_graphs_filtered/' + file + '.ttl')

**Create train, test, val sets**

In [ ]:
def get_graph_type(s):
    s = s.split('/')[-1]
    s = s.split('_')[:-1]
    s = "_".join(s)
    return s

In [ ]:
def get_files_df(INPUT_GRAPHS_FOLDER, INFERENCE_GRAPHS_FOLDER):
    logging.info(f"Creating dataframe for {dataset_name} input/inference pairs")
    rdf_files = []
    for input_graph_path in tqdm(sorted(glob(INPUT_GRAPHS_FOLDER + "*"))):
        input_graph_file = os.path.basename(input_graph_path)
        inference_path = INFERENCE_GRAPHS_FOLDER + input_graph_file.replace('.ttl','_inferred.ttl')
        graph_type = get_graph_type(input_graph_path)
        rdf_pair = {"input_graph_file": input_graph_path, "inference_file": inference_path, "graph_type": graph_type}
        rdf_files.append(rdf_pair)
    files_df = pd.DataFrame.from_dict(rdf_files)
    return files_df

In [ ]:
files_df = get_files_df('MyJenaProject/input_filtered/', 'MyJenaProject/output_filtered/')

In [ ]:
# Remove classes for which only one instance exists
df_count = pd.DataFrame(files_df['graph_type'].value_counts())
graph_type_2_keep = df_count[df_count['count'] > 1].index
files_df = files_df[files_df['graph_type'].isin(graph_type_2_keep)]

In [ ]:
def train_validate_test_split(df, train_percent=0.6, validate_percent=0.2, stratify=None, seed=1):
    val_test_percent = 1 - train_percent
    test_percent = (1 - (train_percent + validate_percent))
    test_percent = test_percent / (test_percent + validate_percent)
    if stratify:
        df_train, df_val_test = train_test_split(df, test_size=val_test_percent, random_state=seed,
                                                 stratify=df[stratify])
        df_val, df_test = train_test_split(df_val_test, test_size=test_percent, random_state=seed,
                                           stratify=df_val_test[stratify])
    else:
        df_train, df_val_test = train_test_split(df, test_size=val_test_percent, random_state=seed)
        df_val, df_test = train_test_split(df_val_test, test_size=test_percent, random_state=seed)
    return df_train, df_val, df_test

In [ ]:
rdf_data_train, rdf_data_val, rdf_data_test = train_validate_test_split(files_df,
                                                                        train_percent=0.6,
                                                                        validate_percent=0.2,
                                                                        stratify="graph_type",
                                                                        seed=1)

In [ ]:
def merge_nt_files_with_rdflib(data, output_file, old_iri, new_iri):
    merged_graph = rdflib.Graph()

    nt_files_orig = data.get('input_graph_file', pd.Series(dtype=str))
    nt_files_inferred = data.get('inference_file', pd.Series(dtype=str))
    tbox_path = pd.Series([f'datasets/{dataset_name}_TBOX.owl'])
    
    nt_files = pd.concat([nt_files_orig, nt_files_inferred, tbox_path], ignore_index=True)

    for nt_file in nt_files:
        try:
            graph = rdflib.Graph()
            if 'TBOX' in nt_file: graph.parse(nt_file)
            else: graph.parse(nt_file, format="turtle")
            if old_iri and new_iri:
                for s, p, o in graph:
                    s = URIRef(str(s).replace(old_iri, new_iri)) if isinstance(s, URIRef) else s
                    p = URIRef(str(p).replace(old_iri, new_iri)) if isinstance(p, URIRef) else p
                    o = URIRef(str(o).replace(old_iri, new_iri)) if isinstance(o, URIRef) else o
                    merged_graph.add((s, p, o))
            else:
                merged_graph += graph  
        except Exception as e:
            print(f"Warning: Could not parse {nt_file} - {e}")

    merged_graph.serialize(destination=output_file)
    print(f"Merged file created at {output_file}")

In [ ]:
merge_nt_files_with_rdflib(rdf_data_train, f'datasets/bin/{dataset_name}_train.owl', 
              'https://kracr.iiitd.edu.in/OWL2Bench', 'https://kracr.iiitd.edu.in/OWL2Bench_train')
merge_nt_files_with_rdflib(rdf_data_val, f'datasets/bin/{dataset_name}_val.owl', 
              'https://kracr.iiitd.edu.in/OWL2Bench', 'https://kracr.iiitd.edu.in/OWL2Bench_val')
merge_nt_files_with_rdflib(rdf_data_test, f'datasets/bin/{dataset_name}_test.owl', 
              'https://kracr.iiitd.edu.in/OWL2Bench', 'https://kracr.iiitd.edu.in/OWL2Bench_test')

**Add noise to training set**

In [ ]:
experiments = get_experimets(dataset_name)

In [ ]:
for experiment in experiments[1:]: 
    dataset_name = experiment['dataset_name']
    file_name = experiment['file_name']
    format_ = experiment['format_']
    add_noise = experiment['add_noise']  

    g_train = rdflib.Graph()
    g_train.parse(f'datasets/bin/{dataset_name}_train.owl', format='turtle')
    print(f'# G_train: {len(g_train)}')

    g_noise = rdflib.Graph()
    g_noise.parse(f'datasets/noise/{file_name}.owl')
    print(f'# G_noise: {len(g_noise)}')

    g_train += g_noise
    g_train.serialize(destination=f'datasets/bin/{file_name}_train.owl')
    print(f'# G_train + G_noise: {len(g_train)}')

    print()